# Introduction to Julia, Jupyter, and ITensor

This is a Jupyter notebook:  it is an interactive interface with Julia, which is similar to Mathematica.  

There are two types of cells: *Markdown* and *code*.  Markdown cells understand $\LaTeX$ -- and you can insert arbitrary html commants in them -- meaning that if you upload an image to your Google Drive, and make it world readible, you can insert it into the cell.  Markdown cells also accept standard *Markdown* syntax.

In code cells, but not markdown cells, you can type Greek letters by typing their LaTeX name -- such as `\alpha` and then hit`tab`.  Subscripts, $f_j$, can be written as `f\_j` followed by `tab`.

Click on a markdown cell to edit it -- typeset it with a "shift-enter".  To run a code cell type shift-enter on it. 

A typical workflow is that you will use ba notebook to interactively write and debug your code.  It is good practice to then turn that code into a function -- which you can run for production.  Often you will put that production code in a "whatever.jl" file, and run it with "julia whatever.jl".  You can also do production runs within Jupyter.  There is another system, called "Jupyterlab" which some people like the workflow of better.  You can also use Jupyter in conjunction with some IDE's.

Here are some useful links:
- [Getting Started with Julia](https://computationalthinking.mit.edu/Spring21/basic_syntax/)
- [Fasttrack to Julia](https://juliadocs.github.io/Julia-Cheat-Sheet/)
- [MATLAB-Julia-Python comparison](https://cheatsheets.quantecon.org/)
- [Plots.jl cheatsheet](https://github.com/sswatson/cheatsheets/blob/master/plotsjl-cheatsheet.pdf)

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Hit shift-enter on each of these lines sequentially to run the code.  Then change $\alpha$ to $\beta$, change the numbers, and rerun**  

In [1]:
α=1

1

In [2]:
α+3

4

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

## Getting Help

Want to know what a symbol means?  For example, what is an `Array`?  Type `?Array` and then shift-enter.

In [ ]:
?Array

# Matrix Product States

Here is a light-weight object to help us manipulate and visualize matrix product states -- further in the notebook we will use the ITensor library -- but even so, it will be helpful to have this.  

In [3]:
# Don't worry too much about this code -- just hit shift-enter on the cell
#
#Import commands to overload
import Base: *,+,-,/,keys,iterate,length,getindex,show,repr,zero
#
# Define ket class
"""ket is a formal representation of a quantum state.

Instantiate with ket(statename::String)

You can add them together and multiply by numbers

Multiplying two kets formally gives the outer product
"""
mutable struct ket <:Number
    storage::Dict
end
#
# Singleton constructor
ket(state)=ket(Dict(string(state)=>1))
#
# Empty state
zero(::ket)=ket(Dict())
ket()=ket(Dict())
#
# Iteration and indexing
iterate(k::ket)=iterate(k.storage)
iterate(k::ket,n)=iterate(k.storage,n)
length(k::ket)=length(k.storage)
keys(k::ket) = keys(k.storage)
getindex(k::ket,val)=getindex(k.storage,val)
#
# Multiplication as outer product
*(k1::ket,k2::ket) = ket(Dict([v1*v2=>s1*s2 for (v1,s1) in k1 for (v2,s2) in k2 ]))
# Multiplication with scalar
*(c::Number,k::ket)=ket(Dict([v=>c*s for (v,s) in k]))
*(k::ket,c::Number)=c*k
/(k::ket,c::Number)=(1/c)*k
#
# Sums
function +(k1::ket,k2::ket) 
    d=copy(k1.storage)
    for key in keys(k2) 
        if (key in keys(k1))
            d[key]+=k2[key]
        else
            d[key]=k2[key]
        end
        if d[key]===zero(d[key])
            delete!(d,key)
        end
    end
    return ket(d)
end
function -(k1::ket,k2::ket) 
    d=copy(k1.storage)
    for key in keys(k2) 
        if (key in keys(k1))
            d[key]-=k2[key]
        else
            d[key]=-k2[key]
        end
        if d[key]===zero(d[key])
            delete!(d,key)
        end
    end
    return ket(d)
end
# Displaying functions
function show(io::IO,::MIME"text/latex",k::ket)
    len=length(k)
    j=1
    outputstring="\$\$"
    for (label,val) in k
        if val===1
            if j>1
                outputstring *="+"
            end
        else
            try
                sn=sign(val)
                if sn>0
                    if j>1
                        outputstring *="+"
                    end    
                    outputstring*=string(val)
                elseif sn<0
                    outputstring*=string(val)
                else
                    if j>1
                        outputstring *="+"
                    end
                    outputstring*="("*string(val)*")"
                end
            catch er #probably a comparison with zero problem
                if j>1
                        outputstring *="+"
                end
                outputstring*="("*string(val)*")"
            end
        end
        outputstring*="|"*string(label)*"\\rangle"
        j+=1
    end
    outputstring*="\$\$"
    print(io,outputstring)
end
function repr(k::ket)
    len=length(k)
    j=1
    outputstring=""
    for (label,val) in k
        if val===1
            if j>1
                outputstring *="+"
            end
        else
            try
                sn=sign(val)
                if sn>0
                    if j>1
                        outputstring *="+"
                    end    
                    outputstring*=string(val)
                elseif sn<0
                    outputstring*=string(val)
                else
                    if j>1
                        outputstring *="+"
                    end
                    outputstring*="("*string(val)*")"
                end
            catch er #probably a comparison with zero problem
                if j>1
                        outputstring *="+"
                end
                outputstring*="("*string(val)*")"
            end
        end
        outputstring*="|"*string(label)*"⟩"
        j+=1
    end
    return outputstring
end
function show(io::IO,k::ket)
    outputstring=repr(k)
    print(io,outputstring)
end
# self-type conversion
ket(k::ket)=k
# alternate display
function tabledisplay(k::ket)
    for pair in k
        println(pair[1]*" "*string(pair[2]))
    end
end

tabledisplay (generic function with 1 method)

Here are some examples of using this

In [4]:
up=ket("↑")

|↑⟩

In [5]:
dn=ket("↓")

|↓⟩

In [ ]:
phi=(1.0 +3.0im)up-0.2dn

In [ ]:
psi=-0.5up+0.4dn

Alternate display format:

In [ ]:
tabledisplay(psi)

We can do outer products

In [ ]:
up*up*up+dn*dn*dn

In [ ]:
tabledisplay(up*up*up+dn*dn*dn)

You can use LaTeX commands in the names.  For example, you can use that to make colored arrows

In [ ]:
ket("\\color{red}\\uparrow")*ket("\\color{blue}\\downarrow")

The whole point of doing this is that I want to be able to make vectors/matrices of these states, and multiply out matrix product states.  For example, here is how you make a Bell state.  In Julia, `*` is matrix multiplication.

In [ ]:
[ up dn ]*[up ; dn]

In [ ]:
[ up dn ]

In [ ]:
[up ; dn]

We can do a GHZ state as

In [ ]:
GHZ=[up dn]*[up 0; 0 dn]*[up 0; 0 dn]*[up 0; 0 dn]*[up ; dn]

Note:  That returns a length-1 vector of kets.  To turn it into a `ket` we need to use the keyword `only`.  So if we want to use `tabledisplay` we need to write:

In [ ]:
tabledisplay(only(GHZ))

We can also do spin-1 states, where the basis is $|0\rangle,|+\rangle,|-\rangle$.

In [ ]:
wf1=ket(-)ket(+)-2ket(0)ket(0)

In [ ]:
tabledisplay(wf1)

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Create and multiply matrices to generate the W state 
$$|W\rangle=|\uparrow\downarrow\downarrow\downarrow\downarrow\rangle
+ |\downarrow\uparrow\downarrow\downarrow\downarrow\rangle
+ |\downarrow\downarrow\uparrow\downarrow\downarrow\rangle
+ |\downarrow\downarrow\downarrow\uparrow\downarrow\rangle
+|\downarrow\downarrow\downarrow\downarrow\uparrow\rangle
$$**

**We will encounter the AKLT state -- which is described by a MPS of the form $\psi=ABBBBB\cdots C$, with
$A=\left(
\begin{array}{cc}
\frac{-1}{\sqrt{3}} |0\rangle & \sqrt{\frac{2}{3}}|+\rangle
\end{array}
\right)$, 
$B=\left(
\begin{array}{cc}
\frac{-1}{\sqrt{3}} |0\rangle & \sqrt{\frac{2}{3}}|+\rangle\\
- \sqrt{\frac{2}{3}}|-\rangle&\frac{1}{\sqrt{3}} |0\rangle
\end{array}
\right)$,
$C=\left(
\begin{array}{cc}
\frac{-1}{\sqrt{3}} |0\rangle \\
- \sqrt{\frac{2}{3}}|-\rangle
\end{array}
\right)$.  Construct these matrices, and multiply them out to produce the 5-site AKLT state.  This should illustrate the compactness of the MPS notation.  It takes fewer numbers to encode the matrices than it does to encode their product.**

**Which basis state has the largest square amplitude?  You will problably find that `tabledisplay` helps for parsing things.**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Symbolic Arithmetic

In [6]:
using Symbolics

In [7]:
# Routines to get our kets to act properly here
*(c::Num,k::ket)=ket(Dict([v=>c*s for (v,s) in k]))
*(k::ket,c::Num)=c*k
/(k::ket,c::Num)=(1/c)*k

/ (generic function with 299 methods)

In [8]:
@variables a,b,c,d,e # defines symbolic variables

5-element Vector{Num}:
 a
 b
 c
 d
  e

In [9]:
empty=ket("⭕️") #represents an empty site
full=ket("🔵") #represents a full site
kz=ket() # the 'zero' ket

In [10]:
mps1=[empty a*full]*[empty b*full;kz empty]*[empty c*full;kz empty]*[d*full ; empty]

1-element Vector{ket}:
 (b)|⭕️🔵⭕️⭕️⟩+(d)|⭕️⭕️⭕️🔵⟩+(c)|⭕️⭕️🔵⭕️⟩+(a)|🔵⭕️⭕️⭕️⟩

In [11]:
tabledisplay(mps1[1])

⭕️🔵⭕️⭕️ b
⭕️⭕️⭕️🔵 d
⭕️⭕️🔵⭕️ c
🔵⭕️⭕️⭕️ a


<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Generate and multiply out this matrix product state that was introduced in class.  Verify that it is a representation of a generic hard-core 2-body wavefunction**
\begin{eqnarray}
|\psi\rangle &=& \left( \begin{array}{cc} |⭕️\rangle &  |🔵\rangle\end{array}
\right)
\left( \begin{array}{cccc} |⭕️\rangle & |🔵\rangle\\
&&|⭕️\rangle&\psi_{12} |🔵\rangle
\end{array}
\right)
\left( \begin{array}{ccccc} |⭕️\rangle &|🔵\rangle\\
&&|⭕️\rangle &&\psi_{23}|🔵\rangle\\
&&&|⭕️\rangle &\psi_{13}|🔵\rangle\\
&&&&|⭕️\rangle \\
\end{array}
\right)\\
&&\times
\left( \begin{array}{ccccc} |🔵\rangle\\
&|⭕️\rangle& &&\psi
_{34}|🔵\rangle\\
&&|⭕️\rangle&&\psi_{24}|🔵\rangle\\
&&&|⭕️\rangle&\psi_{14}|🔵\rangle\\
&&&&|⭕️\rangle
\end{array}
\right)
\left( \begin{array}{cc} \psi_{45} |🔵\rangle\\
\psi_{35}|🔵\rangle\\
\psi_{25}|🔵\rangle\\
\psi_{15}|🔵\rangle\\
|⭕️\rangle
\end{array}
\right)
\end{eqnarray}

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

<h1 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:blue; color:white;" >Checkpoint 1</h1>

# ITensor

ITensor is a package for manipulating tensors. Everything we do in this class can be done with boring out-of-the-box arrays, but this package allows our computer syntax to better match how we write things on paper. This means fewer bugs, and the code is faster to write.

We first load in ITensors

In [12]:
using ITensors

In [13]:
ITensors.enable_debug_checks()

## Basics of ITensor

### Indices

To define a tensor, we first need to make the labeled indices. We will make two indices: i and j, that both take on the values 1 and 2

In [ ]:
i=Index(2,"i")

In [ ]:
j=Index(2,"j")

The label `"i"` and `"j"` are un-necessary. They are useful for readability though. It will let us keep track of the indices

The `Index` objects know how many values they can take on `dim=2`, they have a unique id `id=??` and they have a set of 'tags' -- in this case `i` and `j`.

The purpose of the `Index` is that it will represent one of the legs on a tensor.  For example, we might have a vector with one leg
    
      i
    A---
    
I have labeled the leg `i` to indicate what space the vector lives in.  

A 2x2 matrix could instead be

      i       j
    ---- B ------
    
We can loop over the "values" of an index with

In [ ]:
for val in i
    println(val)
end

These tell both what kind of index we are setting, and what it's value is

### Constructing Tensors

We can then make a 2x2 matrix,
$$\sigma_z=\left(\begin{array}{cc}1&0\\0&-1\end{array}\right)$$

The column index will be `i`, and the row index will be `j`.  We can use the `@show` command to give us the representation of the tensor

In [ ]:
sigz=ITensor(i,j)
sigz[i=>1,j=>1]=1
sigz[i=>2,j=>2]=-1
@show sigz;

Here is how to parse that readout:

"ord=2" -- rank 2 tensor

"Dim 1:" -- Description of first index

  "(dim=2|id=??|"i")" -- the "dim" is the number of elements that index runs over, and "id" is a number which is assigned to each index,  If you scroll up to where we defined "i" you will see that this was the id assigned to that variable.  

"NDTensors.Dense{Int64, Vector{Int64}}" tells about the internal storage

The rest should be clear as a representation of the matrix

The notation should be reminiscent of writing

        A(i=1,j=1)=1
        A(i=2,j=2)=-1

We can also extract the array and the indices separately

In [ ]:
array(sigz) 

In [ ]:
inds(sigz)

### Alternative construction

Here is $\sigma_x$ -- illustrating an alternative way to construct tensors.  (Clearly you do not need the @show -- this  just prints out the details)

In [ ]:
@show sigx=ITensor([0 1; 1 0],i,j)

And $\sigma_y$ -- this example also shows how to input complex numbers.

In [ ]:
@show sigy=ITensor([0 -1im; 1im 0],i,j)

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Make a rank-3 ITensor object which represents**
$$A=\left(\begin{array}{cc}1. |\uparrow\rangle&0.1 |\downarrow\rangle\\
0.1|\uparrow\rangle&1. |\downarrow\rangle\end{array}
\right)$$

**You will need to create a third index object.  Give it a sensible name like `spin` or `mz` so your code remains readible.**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### A simple visualization tool

For rank 3 tensors (which we will use a lot) we can make a simple visualization tool

For example, consider the rank three tensor:
$R=\left(\begin{array}{cc}|\uparrow\rangle&|\downarrow\rangle\\
|\downarrow\rangle&|\uparrow\rangle\end{array}\right)$

We can generate it with

In [14]:
leftindex=Index(2,"left")
rightindex=Index(2,"right")
spinindex=Index(2,"spin")
R=ITensor(0.,leftindex,rightindex,spinindex)
R[leftindex=>:,rightindex=>:,spinindex=>1]=[1 0;0 1]
R[leftindex=>:,rightindex=>:,spinindex=>2]=[0 1;1 0]
@show R

R = ITensor ord=3
Dim 1: (dim=2|id=572|"left")
Dim 2: (dim=2|id=85|"right")
Dim 3: (dim=2|id=489|"spin")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2
[:, :, 1] =
 1.0  0.0
 0.0  1.0

[:, :, 2] =
 0.0  1.0
 1.0  0.0


ITensor ord=3 (dim=2|id=572|"left") (dim=2|id=85|"right") (dim=2|id=489|"spin")
NDTensors.Dense{Float64, Vector{Float64}}

Note -- you may want to look carefully at the slick *slicing* notation that I used to input the tensor:  I told it what the spin-up component was, and what the spin-down component was

Here is a quick routine to pretty-print this tensor in a simpler form  (pp stands for "pretty print").  You done need to worry about the details of the code -- but it is pretty simple

In [15]:
"""
    simplepp(t::ITensor,localbasis::Array,leftind::Index,rightind::Index,siteind::Index)

`simplepp` takes a rank 3 itensor `t`, and a vector of kets `localbasis`, 
and returns an array of kets which represents the same tensor.

You also need to tell it what the indices are

There is no error checking, so use at your own risk.

Example:
```julia-repl
    leftindex=Index(2,"left")
    rightindex=Index(2,"right")
    spinindex=Index(2,"spin")
    R=ITensor(0.,leftindex,rightindex,spinindex)
    R[leftindex=>:,rightindex=>:,spinindex=>1]=[1 0;0 1]
    R[leftindex=>:,rightindex=>:,spinindex=>2]=[0 1;1 0]
    pptensor(R,[ket("↑") ket("↓")],leftindex,rightindex,spinindex)
```

"""
function simplepp(t::ITensor,localbasis::Array,leftind::Index,rightind::Index,siteind::Index)
    Rp=fill(ket(),(dim(leftind),dim(rightind))) # construct an empty array
    for l in leftind
        for r in rightind
            for s in siteind
                Rp[l[2],r[2]]+=t[l,r,s]*localbasis[s[2]]
            end
        end
    end
    Rp
end

simplepp

In [16]:
simplepp(R,[ket("↑") ket("↓")],leftindex,rightindex,spinindex)

2×2 Matrix{ket}:
 1.0|↑⟩  1.0|↓⟩
 1.0|↓⟩  1.0|↑⟩

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Generate an ITensor which represents the matrix
$$B=\left(
\begin{array}{cc}
\frac{-1}{\sqrt{3}} |0\rangle & \sqrt{\frac{2}{3}}|+\rangle\\
- \sqrt{\frac{2}{3}}|-\rangle&\frac{1}{\sqrt{3}} |0\rangle
\end{array}
\right)$$**

**Then use `pptensor` to pretty-print it***

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Inner and Outer Products

Matrix multiplication uses the standard muliplication symbol, "*".  Repeated indices are contracted, while unique indices result in an outer product.

For example, if you had two vectors

      i         i
    A---      B----

Multiplying them would give a scalar

    A--B

Conversely, if you had

      i          j
    A---       C---

Multiplying them would give the outer product

     i     j
    ---A C---

In [ ]:
@variables a1,a2,b1,b2,c1,c2;

In [ ]:
A=ITensor([a1 a2],i)
B=ITensor([b1 b2],i)
C=ITensor([c1 c2],j);

In [ ]:
@show A*B;

In [ ]:
@show A*C;

Note `A*B` was a 'order zero' tensor -- which is equivalent to a scalar.  We can strip all of the `ITensor` features with `only`

In [ ]:
only(A*B)

which should be contrasted with

In [ ]:
A*B

Here is an example with numbers. We construct vectors $v_1=(1,2)$ and $v_2=(3,4)$, we can form their outer product

$\left(\begin{array}{c}1\\2\end{array}\right)\left(\begin{array}{cc}3&4\end{array}\right)=\left(\begin{array}{cc}3&4\\6&8\end{array}\right)$
Recall:  You can type subscripts as "v\_1" then hit tab

In [ ]:
v₁=itensor([1 2],i)
v₂=itensor([3 4],j)
v12=v₁*v₂
@show v12

The same works for higher rank objects.  Consider the following rank 2 tensors

     i   j       i   j      j   k
    ---M---     ---N---    ---L---

    
`M*N` will give a scalar -- as both the `i` and `j` indices will be contracted


      --M--
    i|     |j
      --N--

You can think of it like Einstein notation:
$$M*N=\sum_{ij} M_{ij} N_{ij}$$

`M*L` is instead a rank-2  tensor

     i       k
    ---M---L---


In [ ]:
@variables m11,m12,m21,m22,n11,n12,n21,n22,l11,l12,l21,l22;

In [ ]:
k=Index(2,"k")
M=ITensor([m11 m12;m21 m22],i,j)
N=ITensor([n11 n12;n21 n22],i,j)
L=ITensor([l11 l12;l21 l22],j,k);

In [ ]:
@show M*N

In [ ]:
@show M*L

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Create itensors which represent the vectors (1,1) and (1,-1) -- and construct their dot-product.  These are rank 1 tensors, so they have only one index.  The indices should be the same so that it is contracted.**

**Now construct the outer product of the same two vectors -- you will need to redefine one of them to have a different index**

**Find ${\rm Tr} \sigma_z^2$ by contracting two $\sigma_z$ matrices**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Changing indices of an existing tensor

If we want to multiply two matrices to get a third one we want the indices to look like:

$$C_{ij}=\sum_j A_{ik} B_{kj}$$

It would be very unweildy if we needed to reconstruct our tensors to get the indices right.  There is, of course, a function to change the index.  Lets take the Pauli matrices that we defined, and try to calculate the commutator os $\sigma_x$ and $\sigma_y$.  

As a first step, I'll just refresh the definitions, to make sure that they have not been modified in the notebook.

In [ ]:
i=Index(2,"i")
j=Index(2,"j")
sigz=ITensor([1 0;0 1],i,j)
sigx=ITensor([0 1;1 0],i,j)
sigy=ITensor([0 -1im;1im 0],i,j)

Next we need the new index that we will contract over

In [ ]:
k=Index(2,"k") #Generate a new index

There are two ways of changing an index.  The first is in-place

In [ ]:
replaceinds!(sigx,j=>k)
@show sigx

The second way is to create a new view

In [ ]:
newsigy=replaceinds(sigy,i=>k)

We can see that the original is unchanged

In [ ]:
sigy

We can now multiply the two

In [ ]:
@show sigx*newsigy

We can also do this inline

In [ ]:
sigz=ITensor([1 0;0 1],i,j)
sigx=ITensor([0 1;1 0],i,j)
sigy=ITensor([0 -1im;1im 0],i,j)
@show replaceinds(sigx,j=>k)*replaceinds(sigy,i=>k)

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

Calculate the commutator of $\sigma_x$ and $\sigma_y$ using ITensors

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Prime levels

If we have a bunch of indices, which are really just copies of the same thing, it is good to have notation to suggest that.  For example, in the Pauli matrices, `i`, `j`, and `k` had the same meaning.  The label was just to know which to contract.  In such cases, rather than introducing new indices, we can use 'primes' -- which you indicate by simply putting a tick mark after the index

In [ ]:
@show mat1=itensor([0 1;0 0],i,i')

We can create views with changed prime levels

In [ ]:
mat1b=replaceprime(mat1,1=>3)

This leaves mat1 unchanged

In [ ]:
mat1

Finally we can add or subtract some number of prime levels from all indices

In [ ]:
prime(mat1)

In [ ]:
mat2=prime(mat1,3)

In [ ]:
mat3=prime(mat2,-1)

There is also an in-place version

In [ ]:
prime!(mat2,-2)
mat2

This makes matrix multiplication easy

In [ ]:
sigx=ITensor([0 1;1 0],i,i')
sigy=ITensor([0 -1im;1im 0],i,i')
prodxy=replaceprime(sigx*prime(sigy),2=>1)
@show prodxy

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Again calculate the commutator of $\sigma_x$ and $\sigma_y$ -- but this time using primes** 

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End 
   Problems</h3>

<h1 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:blue; color:white;" >Checkpoint 2</h1>

### Singular value decomposition

Given a wavefunction, say $|\psi\rangle=|\uparrow\uparrow\uparrow\uparrow\downarrow\rangle+|\uparrow\uparrow\uparrow\downarrow\uparrow\rangle+|\uparrow\uparrow\downarrow\uparrow\uparrow\rangle+
|\uparrow\downarrow\uparrow\uparrow\uparrow\rangle+|\downarrow\uparrow\uparrow\uparrow\uparrow\rangle$, we want to write this as
$$
|\psi\rangle=\sum_\alpha \lambda_\alpha |\alpha\rangle_{12} |\alpha\rangle_{345} 
$$
where $|\alpha\rangle_{12}$ is a state involving sites 1 and 2, while $|\alpha\rangle_{345}$ involves sites 3,4,5.

The states $|\alpha\rangle_{12}$ and $|\beta\rangle_{345}$ act on different Hilbert spaces and cannot be compared.  Those acting on the same space should be orthonormal:
$_{12}\langle\alpha|\beta\rangle_{12}=\delta_{\alpha\beta}$ and 
$_{345}\langle\alpha|\beta\rangle_{345}=\delta_{\alpha\beta}$.

This is referred to as the Schmidt decomposition.  The non-negative real coefficients $\lambda_\alpha$ are the Schmidt values, and they define what is meant by entanglement.  The {\em entanglement entropy} is defined as
$$
S=-\sum_\alpha \lambda_\alpha^2\ln  \lambda_\alpha^2.
$$
It tells you how much information you get about the right half of the system from measuring the left.

The mathematical operation to calculate the Schmidt decomposition is the Singular Value Decomposition

Lets generate this state.  First we need the index objects for each of the five sites:

In [18]:
spinindices=[Index(2,"site,"*string(j)) for j in 1:5]

5-element Vector{Index{Int64}}:
 (dim=2|id=300|"1,site")
 (dim=2|id=695|"2,site")
 (dim=2|id=86|"3,site")
 (dim=2|id=453|"4,site")
 (dim=2|id=817|"5,site")

In [19]:
psi5=ITensor(spinindices)

ITensor ord=5 (dim=2|id=300|"1,site") (dim=2|id=695|"2,site") (dim=2|id=86|"3,site") (dim=2|id=453|"4,site") (dim=2|id=817|"5,site")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [20]:
indlist=[spinindices[j]=>1 for j in 1:5]
@show (indlist)

indlist = Pair{Index{Int64}, Int64}[(dim=2|id=300|"1,site") => 1, (dim=2|id=695|"2,site") => 1, (dim=2|id=86|"3,site") => 1, (dim=2|id=453|"4,site") => 1, (dim=2|id=817|"5,site") => 1]


5-element Vector{Pair{Index{Int64}, Int64}}:
 (dim=2|id=300|"1,site") => 1
 (dim=2|id=695|"2,site") => 1
  (dim=2|id=86|"3,site") => 1
 (dim=2|id=453|"4,site") => 1
 (dim=2|id=817|"5,site") => 1

In [21]:
indlist=[spinindices[j]=>1 for j in 1:5] # make a list which sets the spin indices
for s in 1:5 # s is the site which is down
    for t in 1:5
        if t==s
            indlist[t]=(spinindices[t]=>2)
        else
            indlist[t]=(spinindices[t]=>1)
        end
    end 
    psi5[indlist...]=1
end 

Our regular way of displaying the wavefunction is uninformative, so lets write a new pretty printing command.  For simplicity this command assumes that the indices of `T` are all identical site indices, and they are arranged in the order of the sites.

In [17]:
function scalarpp(T::ITensor,localstates::Array{ket})
    ar=array(T) #unpack raw tensor -- as it is easier to work with
    result=ket() #empty ket to use as result
    for ci in CartesianIndices(ar) #loop over indices of the array
        nonzero=true
        try
            if abs(ar[ci])<1E-6 # only do something if the array's entry is above a threshold
                nonzero=false
            end
        catch
            nonzero=true
        end
        if nonzero
            newterm=ket("") # create trivial ket
            for site in ci.I
                newterm=newterm*localstates[site]  # add site information to ket
            end
            result+= ar[ci]*newterm # add ket to result
        end
    end
    result
end

scalarpp (generic function with 1 method)

In [22]:
scalarpp(psi5,[ket("↑"),ket("↓")])

|↓↑↑↑↑⟩+|↑↓↑↑↑⟩+|↑↑↓↑↑⟩+|↑↑↑↓↑⟩+|↑↑↑↑↓⟩

We can perform an SVD

In [ ]:
(u,s,v)=svd(psi5,spinindices[1:2]);

The Schmidt values are stored in s

In [ ]:
@show s

In [ ]:
s[1,1]^2+s[2,2]^2

If our state was properly normalized, the squares of the Schmidt values should add to 1.  Normalizing them

In [ ]:
normedSchmidts=[s[1,1]/sqrt(5),s[2,2]/sqrt(5)]

the indices `'Link,u'` and `'Link,v'` are the $\alpha$'s -- they label the basis states on each part of the system

Here are the basis states for the left two sites

In [ ]:
uind=commonindex(u,s)

In [ ]:
u1=u*onehot(uind=>1)
scalarpp(u1,[ket("↑"),ket("↓")])

In [ ]:
u2=u*onehot(uind=>2)
scalarpp(u2,[ket("↑"),ket("↓")])

And here are the basis states for the right 3

In [ ]:
vind=commonindex(v,s)

In [ ]:
v1=v*onehot(vind=>1)
v1k=scalarpp(v1,[ket("↑"),ket("↓")])

In [ ]:
v2=v*onehot(vind=>2)
v2k=scalarpp(v2,[ket("↑"),ket("↓")])

In [ ]:
tabledisplay(v1k)

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Find the entanglement entropy of that state when we break it between the second and third site.  (This just requires using the normalized Schmidt values.)**

**Find the entanglement entropy if you break it between the first and second site**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:yellow; color:black;" >Aside: More visualization</h3>

Here are functions to pretty print Tensors with any number of site indices, and either one or two bond indices.

In [23]:
vectorpp(T::ITensor,bondindex::Index,localstates::Array{ket})=
    [scalarpp(T*onehot(bondval),localstates) for bondval in eachindval(bondindex)]

vectorpp (generic function with 1 method)

In [24]:
matrixpp(T::ITensor,leftbondindex::Index,rightbondindex::Index,localstates::Array{ket})=
    [scalarpp(T*onehot(leftbondval,rightbondval),localstates) for 
            leftbondval in eachindval(leftbondindex),
            rightbondval in eachindval(rightbondindex)]

matrixpp (generic function with 1 method)

In [25]:
vectorpp(u,uind,[ket("↑") ket("↓")])

LoadError: UndefVarError: `u` not defined

In [26]:
vectorpp(v,vind,[ket("↑") ket("↓")])

LoadError: UndefVarError: `v` not defined

In [27]:
leftindex=Index(2,"left")
rightindex=Index(2,"right")
spinindex=Index(2,"spin")
R=ITensor(0.,leftindex,rightindex,spinindex)
R[leftindex=>:,rightindex=>:,spinindex=>1]=[1 0;0 1]
R[leftindex=>:,rightindex=>:,spinindex=>2]=[0 1;1 0]
matrixpp(R,leftindex,rightindex,[ket("↑") ket("↓")])

2×2 Matrix{ket}:
 1.0|↑⟩  1.0|↓⟩
 1.0|↓⟩  1.0|↑⟩

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:yellow; color:black;" >End Aside</h3>

### Constructing Matrix Product States from arbitrary wavefunctions

We now have the tools required to convert `psi5` into a matrix product state

Step 1 create the far left matrix, A1 by doing a SVD on the entire state

In [ ]:
(u1,s1,v1)=svd(psi5,spinindices[1])
uind1=commoninds(u1,s1)[1]
swaptags!(u1,uind1.tags,"link,1") #change tags on link index
swaptags!(s1,uind1.tags,"link,1") #change tags on link index
bond1=settags(uind1,"link,1")
A1=u1
rest1=s1*v1;

In [ ]:
vectorpp(A1,bond1,[up dn])

In [ ]:
vectorpp(rest1,bond1,[up dn])

Step 2: create A2 by doing a SVD on rest1

In [ ]:
(u2,s2,v2)=svd(rest1,spinindices[2],bond1)
uind2=commoninds(u2,s2)[1]
swaptags!(u2,uind2.tags,"link,2") #change tags on link index
swaptags!(s2,uind2.tags,"link,2") #change tags on link index
bond2=settags(uind2,"link,2")
A2=u2
rest2=s2*v2;

In [ ]:
println(inds(A2))
matrixpp(A2,bond1,bond2,[up dn])

In [ ]:
vectorpp(rest2,bond2,[up dn])

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Keep going, and generate A3, A4, A5.  Clearly it would be easy to encapsulate this, making a function which generates the MPS representation of a generic wavefunction -- but you do not need to make that function (unless you think it would be fun)**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Transfer Matrices

In calculating expectation values of matrix product states we need to construct *transfer matrices*.  For example imagine that we have a rank 3 tensor $$A=\left(\begin{array}{cc}1. |\uparrow\rangle&0.1 |\downarrow\rangle\\
0.1|\uparrow\rangle&1. |\downarrow\rangle\end{array}
\right)$$
The transfer matrix is a rank 4 tensor formed by contracting the site indices between two copies of $A$ -- or more correctly a copy of $A$ and a copy of $A^*$,
<img src="https://docs.google.com/drawings/d/e/2PACX-1vSdoSIOG6aebpmhwCLcKJpyEITEQntmegz-W1I0pxt3P9cYadTVfnIKddsEWaT-9wlzX6Vtyz9ZL4JV/pub?w=374&amp;h=448" width=200>

The one new thing that we need is taking complex conjugates.  The command which does that in ITensor is `dag`-- which is short for 'dagger' -- meaning the operation which changes column vectors into row vectors, and simultaneously complex conjugation.  In our formalism there is no difference between row and column vectors, so 'dagger' is simply complex conjugation.  (When we talk about conservation laws it will do some extra things.)

In [ ]:
s=Index(2,"s")
@show psi=ITensor([0.1+0.2im,0.3-0.4im],s)

In [ ]:
@show dag(psi)

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Generate the tensor $$A=\left(\begin{array}{cc}1. |\uparrow\rangle&0.1 |\downarrow\rangle\\
0.1|\uparrow\rangle&1. |\downarrow\rangle\end{array}
\right)$$ and use it to construct the transfer matrix 
<img src="https://docs.google.com/drawings/d/e/2PACX-1vSdoSIOG6aebpmhwCLcKJpyEITEQntmegz-W1I0pxt3P9cYadTVfnIKddsEWaT-9wlzX6Vtyz9ZL4JV/pub?w=374&amp;h=448" width=200>
as a 2x2x2x2 tensor with indices `l`, `l'`, `r`, `r'`**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Combining Indices

It is often useful to combine indices.  For example, consider a two-spin wavefunction, such as the Bell state 
$|\psi\rangle=|\uparrow\uparrow\rangle+|\downarrow\downarrow\rangle$.  We can think of the coefficients of this wavefunction as a $2\times 2$ matrix
$$
|\psi\rangle =\left(\begin{array}{cc}|\uparrow\rangle& |\downarrow\rangle\end{array}\right)
\left(\begin{array}{cc}1&0\\0&1\end{array}\right)\left(\begin{array}{c}|\uparrow\rangle\\|\downarrow\rangle\end{array}\right)
$$
or a length 4 vector
$$
|\psi\rangle =\left(\begin{array}{cccc}1&0&0&1\end{array}\right)
\left(\begin{array}{c}|\uparrow\uparrow\rangle\\ |\uparrow\downarrow\rangle\\
|\downarrow\uparrow\rangle\\ |\downarrow\downarrow\rangle
\end{array}\right)
$$

Here is an example of that procedure.

In [ ]:
s1=Index(2,"s1")
s2=Index(2,"s2")
@show bell=ITensor([1 0;0 1],s1,s2)

In [ ]:
c12=combiner(s1,s2,tags="s12")
@show newbell=c12*bell

The object  `c12` can be interpreted as a rank 3 tensor with legs `s1`, `s2`, and `s12`

In [ ]:
@show c12

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Use a combiner to combine the `l` and `l'` indices of our transfer matrix.  
<img src="https://docs.google.com/drawings/d/e/2PACX-1vSdoSIOG6aebpmhwCLcKJpyEITEQntmegz-W1I0pxt3P9cYadTVfnIKddsEWaT-9wlzX6Vtyz9ZL4JV/pub?w=374&amp;h=448" width=200>
Also combine the `r` and `r'` indices -- making a $4\times 4$ matrix:
<img src="https://docs.google.com/drawings/d/e/2PACX-1vQ-jKt1qXoXR_b0TE2TJTGlzV6Y4V0wgDUilRnRceG6ANVulu2fDmDBVdZegQGH6pFNX11Y0lt-pjZQ/pub?w=1431&amp;h=653" width=400>**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Functions

There are a number of ways of writing functions in Julia.  Here is one notation, that will generate a rotation matrix. 

In [ ]:
function rotmat(angle)
    i=Index(2,"i")
    ITensor([cos(angle) sin(angle) ; -sin(angle) cos(angle)],i,i')
end

In [ ]:
@show rotmat(pi/4)

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Write a function which is called with a number, `eps` -- and returns the $4\times4$ transfer matrix corresponding to $A=\left(\begin{array}{cc}|\uparrow\rangle&\epsilon |\downarrow\rangle\\
\epsilon |\uparrow\rangle&|\downarrow\rangle\end{array}\right)$.**

**Verify that it works by checking to see if it gives the same answer as we got when we set $\epsilon=0.1$.**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Symbolic Transfer Matrix

We can also symbolically manipulate ITensors

In [28]:
@variables ϵ

1-element Vector{Num}:
 ϵ

In [29]:
l=Index(2,"l")
r=Index(2,"r")
s=Index(2,"s")
As=ITensor(l,r,s)

ITensor ord=3 (dim=2|id=108|"l") (dim=2|id=526|"r") (dim=2|id=375|"s")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [30]:
As[l=>1,r=>2,s=>2]=ϵ #Do this assignment first, so that it sets "Type" appropriately
As[l=>2,r=>1,s=>2]=ϵ
As[l=>1,r=>1,s=>1]=1
As[l=>2,r=>2,s=>1]=1
matrixpp(As,l,r,[up dn])

2×2 Matrix{ket}:
 1.0|↑⟩  (ϵ)|↓⟩
 (ϵ)|↓⟩  1.0|↑⟩

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Make a symbolic representation of the $4\times4$ transfer matrix associated with this the tensor we just created -- combining the two left indices, and combining the two right indices.**

**Symbolically construct the $4\times4$ transfer matrix
<img src="https://docs.google.com/drawings/d/e/2PACX-1vTPq2IVjT0j3XrEmXlqt0gOcRqwF_RbXYN53o6yKwWAlJD8PEjP03mScBLBrol_2UxCUpyd5Qgqzh-6/pub?w=638&amp;h=1067" width=200>
-- (but add combiners to write it as a 4x4 matrix).**

**Here $X$ is the $\sigma_x$ operator.**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

### Dense Eigensolver

Consider a system of three spin-1/2's with Hamiltonian $H=\sigma_x^{(1)}+\sigma_x^{(2)}+\sigma_x^{(3)}$.  We will construct the $8\times8$ matrix Hamiltonian, and find its eigenvalues

In [31]:
s1=Index(2,"s1")
s2=Index(2,"s2")
s3=Index(2,"s1")

(dim=2|id=613|"s1")

Here is one construction method

In [32]:
sigx1=ITensor([0 1;1 0],s1,s1')
id1=ITensor([1 0;0 1],s1,s1')
sigx2=ITensor([0 1;1 0],s2,s2')
id2=ITensor([1 0;0 1],s2,s2')
sigx3=ITensor([0 1;1 0],s3,s3')
id3=ITensor([1 0;0 1],s3,s3')
SpinHam1=sigx1*id2*id3+id1*sigx2*id3+id1*id2*sigx3

ITensor ord=6 (dim=2|id=474|"s1") (dim=2|id=474|"s1")' (dim=2|id=985|"s2") (dim=2|id=985|"s2")' (dim=2|id=613|"s1") (dim=2|id=613|"s1")'
NDTensors.Dense{Float64, Vector{Float64}}

As you can see, this creates the Hamiltonian as a 2x2x2x2x2x2 tensor.  Maybe a more conventional representation is to combine all the spin indices together to make an $8\times8$ matrix.

In [33]:
c8=combiner(s1,s2,s3;tags="s123")

ITensor ord=4 (dim=8|id=338|"s123") (dim=2|id=474|"s1") (dim=2|id=985|"s2") (dim=2|id=613|"s1")
NDTensors.Combiner

In [34]:
@show c8*SpinHam1*prime(c8)

c8 * SpinHam1 * prime(c8) = ITensor ord=2
Dim 1: (dim=8|id=338|"s123")
Dim 2: (dim=8|id=338|"s123")'
NDTensors.Dense{Float64, Vector{Float64}}
 8×8
 0.0  1.0  1.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0
 0.0  1.0  0.0  0.0  1.0  0.0  0.0  1.0
 0.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0  1.0  1.0  0.0


ITensor ord=2 (dim=8|id=338|"s123") (dim=8|id=338|"s123")'
NDTensors.Dense{Float64, Vector{Float64}}

Here are the 'labels' for those 3-site basis states.  You can verify that the 1's are in the right places

In [35]:
basis8=kron([up dn],[up dn],[up dn])

1×8 Matrix{ket}:
 |↑↑↑⟩  |↑↑↓⟩  |↑↓↑⟩  |↑↓↓⟩  |↓↑↑⟩  |↓↑↓⟩  |↓↓↑⟩  |↓↓↓⟩

Here is how to diagonalize it:  D will be a diagonal matrix of the eigenvalues, and U will be the matrix which diagonalizes it: $H=U D U^{-1}$ -- or $HU=UD$.  Thus the columns of $U$ are the *right* eigenvectors of $H$.

In [36]:
D1,U1=eigen(SpinHam1,(s1',s2',s3'),(s1,s2,s3));

The eigenvalues are just the net x-magnetization

In [ ]:
diagsize=dims(D1)
[D1[i,i] for i in 1:diagsize[1]]

Here is the wavefunction for the highest energy state:  It corresponds to all spins pointing in the +x direction.  In the z-basis, this means it is a linear combination of all basis states with equal weights.

In [ ]:
linkind=commonind(D1,U1)
tabledisplay(scalarpp(U1*onehot(linkind=>8),[up dn]))

Here is the lowest energy state -- corresponding to all spins in the -x direction

In [ ]:
linkind=commonind(D1,U1)
tabledisplay(scalarpp(U1*onehot(linkind=>1),[up dn]))

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Earlier you wrote a function which would generate the numerical $4\times4$ transfer matrix for a simple infinite matrix product state.  Numerically find its eigenvalues when $\epsilon=0.1$. (The eignevalue solver only works with numerical matrices -- as far as I know there is not a good Julia package for symbolic eigenvalues)**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Types (Object/Structs)

In addition to knowing how to construct 'functions', the 'verbs' of programming, we need to know how to construct 'objects', the 'nouns'.  We have seen a few nice examples here:  Built-in types like "Arrays", "Complex Numbers", library types like "ITensors" or "Index" objects, or even the "ket" object.  

One thing that it would be useful to have is an object which encapsulates an ITensor, and makes it act like a square matrix:  When you multiply a vector by it, you get a new vector with the same indices.

We first define the container:

In [ ]:
mutable struct it_matrix 
    storage::ITensor
end

Next we define a couple functions (aka methods) which tell how these objects act

In [ ]:
*(M::it_matrix,v::ITensor)=noprime(M.storage*v) # multiplication
(M::it_matrix)(v::ITensor)=M*v  #evaluation

Here is how we create an `it_matrix` object:

In [ ]:
i=Index(2,"i")
sigx=ITensor([0. 1. ; 1. 0.],i,i')
sigxmat=it_matrix(sigx)

And here is an example of multiplying a vector

In [ ]:
vec1=ITensor([1. 0.],i)
@show sigxmat*vec1

We have also defined a method which allows us to use it like

In [ ]:
@show sigxmat(vec1)

Here is a fancier version of the multiply 'method' which does some error checking

In [ ]:
function *(M::it_matrix,v::ITensor)
    Mv=M.storage*v
    if order(Mv) !=order(v)
        error(
            string("Matrix and vector act on different spaces, and cannot be multiplied\n",
                "This may be a problem with using the wrong indices\n\n",
                "M*v indices: $(inds(Mv)) \n\n",
                "v indices: $(inds(v))"),)
    end
    return noprime(Mv)
end

Here is an example which will trigger this error

In [ ]:
j=Index(2,"j")
sigx2=ITensor([0 1;1 0],j,j')
sigxmat*sigx2

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Complete the following code, which will create a generalization of `it_matrix`, which will store an array of `ITensor` objects, and acts like the sum of them.**

**For example, we can use this to make a *sparse* representation of the Hamiltonian $H=\sigma_x^{(1)}+\sigma_x^{(2)}+\sigma_x^{(3)}$ that we worked with recently.  Given a vector $v$, we can construct $Hv$ as the sum $\sigma_x^{(1)} v+\sigma_x^{(2)}v+\sigma_x^{(3)}v$.  Since $H$ is an $8\times 8$ matrix, and $v$ is length $8$ -- the naive multiplication requires 64 multiplications. Multiplying by $\sigma_x^{(1)}$ only requires 16 operations.  Thus the "lazy sum" approach ends up using 48 operations.  For larger matrices this becomes a significant improvement**

In [ ]:
# This is the full container object
mutable struct it_summatrix 
    storage::Array{ITensor}
end
# Define the multiplication function
function *(M::it_summatrix,v::ITensor)
    result=noprime(M.storage[1]*v) # first term in the sum
    numterms=length(M.storage)
    ## now complete the function by looping over the rest of the terms, each time adding
    ## noprime(M.storage[j]*v)
    #
    #
    
    
    #
end
#Define evaluate
(M::it_summatrix)(v::ITensor)=M*v  #evaluation

Test it out on

In [ ]:
s1=Index(2,"s1")
s2=Index(2,"s2")
s3=Index(2,"s1")
sigx1=ITensor([0 1;1 0],s1,s1')
sigx2=ITensor([0 1;1 0],s2,s2')
sigx3=ITensor([0 1;1 0],s3,s3')
lazyH=it_summatrix([sigx1 sigx2 sigx3])
v=ITensor(s1,s2,s3)
v(s1=>1,s2=>1,s3=>1)=1
w=lazyH*v;

In [ ]:
scalarpp(v,[up dn])

In [ ]:
scalarpp(w, [up dn])

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

### Sparse Eigensolver

We are going to be using large tensors/matrices -- and numerically finding all of their eigenvalues will be expensive.  There are, however, efficient iterative routines for calculating just a few eigenvalues and eigenvectors.

In [ ]:
using KrylovKit

First we generate an `it_matrix` object -- which has the right properties to work with the eigensolver

In [ ]:
sigx1=ITensor([0 1;1 0],s1,s1')
id1=ITensor([1 0;0 1],s1,s1')
sigx2=ITensor([0 1;1 0],s2,s2')
id2=ITensor([1 0;0 1],s2,s2')
sigx3=ITensor([0 1;1 0],s3,s3')
id3=ITensor([1 0;0 1],s3,s3')
SpinHam1=sigx1*id2*id3+id1*sigx2*id3+id1*id2*sigx3
it_SpinHam1=it_matrix(SpinHam1);

Next, since it is an iterative method, we need a starting vector.  The closer this is to the desired eigenvector, the better.  Here we will just use a random vector.

In [ ]:
startv=randomITensor(s1,s2,s3)
scalarpp(startv, [up dn])

Here is the syntax for using the solver.  The number `1` asks for at least one eigenvector.  The `:SR` asks for the most negative eigenvector.  `ishermitian=true` tells the algorithm that we are working with a Hermitian matrix, which simplifies some of the arithmetic that it uses.  (Essentially, it can take advantage of the fact that the eigenvectors are orthogonal to one-another)

In [ ]:
vals,vecs,info=eigsolve(it_SpinHam1,startv,1,:SR;ishermitian=true);

Even though we only asked for one eigenvalue, it found 4 of them.  Essentially the other 3 were 'free', so it went ahead and reported them.

In [ ]:
vals

In [ ]:
scalarpp(vecs[1], [up dn])

In [ ]:
info

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >Problems</h3>

**Create a function which when given $J$ and $h$, numerically finds the lowest two energy states of the Hamiltonian:
$H=-J \left[(\sigma_z)_1 (\sigma_z)_2 
+ (\sigma_z)_2 (\sigma_z)_3 
\right] -h\left[(\sigma_x)_1+(\sigma_x)_2+(\sigma_x)_3\right]$.  Make two versions:  One that uses the dense eigensolver, and the other that uses the iterative eigensolver.  Make sure the answers agree at the following three points:  $(J,h)=(1,0),(0,1),(1,1)$.**  

**Note, the iterative eigensolver might have trouble when $h=0$ -- so you may want to also try using a very small $h$. at that point.  The issue has to do with how degeneracies are handled by the algorithm.**

<h3 style="border: 2px solid #000; border-radius: 10px; padding: 20px; background-color:red; color:white;" >End Problems</h3>

# Not in notebook for in-class.   Solutions to Homework 3

**Write a program which will take in the matrix describing a uniform infinite matrix product state, and give you the transfer matrix, and its eigenvalues.  Test it with with the wavefunctions from section~\ref{special} of this chapter.  Verify the single-particle wavefunction yields a non-diagonalizable transfer matrix, and the chiral state yields one with complex eigenvalues.**

In [61]:
function MakeTransfer(tensor::ITensor,leftindex::Index,rightindex::Index,siteindex::Index)
    t=prime(dag(tensor),(leftindex,rightindex))*tensor
    leftcombiner=combiner(leftindex,leftindex',tags="lc")
    rightcombiner=combiner(rightindex,rightindex',tags="rc")
    t*leftcombiner*rightcombiner
end

MakeTransfer (generic function with 1 method)

In [59]:
TAs=MakeTransfer(As,left,right,site)

ITensor ord=2 (dim=4|id=109|"lc") (dim=4|id=715|"rc")
NDTensors.Dense{Num, Vector{Num}}

In [62]:
array(TAs)

4×4 Matrix{Num}:
   1    0    0  ϵ^2
   0    1  ϵ^2    0
   0  ϵ^2    1    0
 ϵ^2    0    0    1

In [63]:
l=Index(2,"l")
r=Index(2,"r")
s=Index(2,"s")
As2=ITensor(l,r,s)

ITensor ord=3 (dim=2|id=318|"l") (dim=2|id=614|"r") (dim=2|id=512|"s")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [64]:
As2[l=>1,r=>2,s=>2]=ϵ #Do this assignment first, so that it sets "Type" appropriately
As2[l=>2,r=>1,s=>1]=ϵ
As2[l=>1,r=>1,s=>1]=1
As2[l=>2,r=>2,s=>2]=1
matrixpp(As2,l,r,[up dn])

2×2 Matrix{ket}:
 1.0|↑⟩  (ϵ)|↓⟩
 (ϵ)|↑⟩  1.0|↓⟩

In [66]:
TAs2=MakeTransfer(As2,l,r,s)

ITensor ord=2 (dim=4|id=897|"lc") (dim=4|id=781|"rc")
NDTensors.Dense{Num, Vector{Num}}

In [67]:
array(TAs2)

4×4 Matrix{Num}:
   1  0  0  ϵ^2
   ϵ  0  0    ϵ
   ϵ  0  0    ϵ
 ϵ^2  0  0    1

In [85]:
function Teig(mat::ITensor)
    leftinds=findinds(mat,"lc")
    rightinds=findinds(mat,"rc")
    D,U=eigen(mat,leftinds,rightinds)
    [D[j,j] for j in 1:dims(D)[1]]
end

Teig (generic function with 1 method)

In [74]:
freeA=onehot(s=>1)*ITensor([1 0;0 1],l,r)+onehot(s=>2)*ITensor([0 0.1;0 0],l,r)

ITensor ord=3 (dim=2|id=512|"s") (dim=2|id=318|"l") (dim=2|id=614|"r")
NDTensors.Dense{Float64, Vector{Float64}}

In [92]:
anA=onehot(s=>2,l=>1,r=>2)*ϵ+onehot(s=>1)*ITensor([1 0;0 1],l,r)

ITensor ord=3 (dim=2|id=512|"s") (dim=2|id=318|"l") (dim=2|id=614|"r")
NDTensors.Dense{Num, Vector{Num}}

In [93]:
matrixpp(anA,l,r,[ket(".") ket("X")])

2×2 Matrix{ket}:
 1.0|.⟩  (ϵ)|X⟩
         1.0|.⟩

In [75]:
matrixpp(freeA,l,r,[ket(".") ket("X")])

2×2 Matrix{ket}:
 1.0|.⟩  0.1|X⟩
         1.0|.⟩

In [94]:
freeT=MakeTransfer(anA,l,r,s)

ITensor ord=2 (dim=4|id=25|"lc") (dim=4|id=23|"rc")
NDTensors.Dense{Num, Vector{Num}}

In [95]:
array(freeT)

4×4 Matrix{Num}:
 1.0  0.0  0.0  ϵ^2
 0.0  1.0  0.0    0.0
 0.0  0.0  1.0    0.0
 0.0  0.0  0.0    1.0

In [86]:
Teig(freeT)

D = ITensor ord=2
Dim 1: (dim=4|id=556|"Link,eigen")'
Dim 2: (dim=4|id=556|"Link,eigen")
NDTensors.Diag{ComplexF64, Vector{ComplexF64}}
 4×4
 1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im


4-element Vector{ComplexF64}:
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im
 1.0 + 0.0im

In [84]:
dims(freeT)[1]

4

In [ ]:
D1,U1=eigen(SpinHam1,(s1',s2',s3'),(s1,s2,s3));

In [37]:
As

ITensor ord=3 (dim=2|id=108|"l") (dim=2|id=526|"r") (dim=2|id=375|"s")
NDTensors.Dense{Num, Vector{Num}}

In [49]:
left=findindex(As,"l")
right=findindex(As,"r")
site=findindex(As,"s")

(dim=2|id=375|"s")

In [52]:
prime(As,(left,right))

ITensor ord=3 (dim=2|id=108|"l")' (dim=2|id=526|"r")' (dim=2|id=375|"s")
NDTensors.Dense{Num, Vector{Num}}

In [56]:
combiner(left,left',tags="lc")

ITensor ord=3 (dim=4|id=263|"lc") (dim=2|id=108|"l") (dim=2|id=108|"l")'
NDTensors.Combiner

In [54]:
tags(left)

"l"

In [98]:
l3=Index(3,"l")
r3=Index(3,"r")
s3=Index(3,"s")
chiralA=onehot(l3=>1,r3=>2,s3=>3)+onehot(l3=>2,r3=>3,s3=>1)+onehot(l3=>3,r3=>1,s3=>2)
matrixpp(chiralA,l3,r3,[ket("A") ket("B") ket("C")])

3×3 Matrix{ket}:
         1.0|C⟩        
                 1.0|A⟩
 1.0|B⟩                

In [99]:
chiralT=MakeTransfer(chiralA,l3,r3,s3)

ITensor ord=2 (dim=9|id=157|"lc") (dim=9|id=989|"rc")
NDTensors.Dense{Float64, Vector{Float64}}

In [100]:
array(chiralT)

9×9 Matrix{Float64}:
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [101]:
Teig(chiralT)

D = ITensor ord=2
Dim 1: (dim=9|id=957|"Link,eigen")'
Dim 2: (dim=9|id=957|"Link,eigen")
NDTensors.Diag{ComplexF64, Vector{ComplexF64}}
 9×9
 -0.4999999999999999 - 0.8660254037844388im                  0.0 + 0.0im                 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                 -0.4999999999999999 + 0.8660254037844388im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                                 0.0 + 0.0im                 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                                 0.0 + 0.0im                 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im                 0.0 + 0.0im
                 0.0 + 0.0im                                 0.0 + 0.0im           

9-element Vector{ComplexF64}:
 -0.4999999999999999 - 0.8660254037844388im
 -0.4999999999999999 + 0.8660254037844388im
                 0.0 + 0.0im
                 0.0 + 0.0im
                 0.0 + 0.0im
                 0.0 + 0.0im
                 0.0 + 0.0im
                 0.0 + 0.0im
  0.9999999999999998 + 0.0im